<a href="https://colab.research.google.com/github/Humagonen/Amazon_reviews_Sentiment_analysis/blob/main/streamlit_web_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Streamlit Sentiment analysis web app

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!pip install -q streamlit tensorflow pandas numpy
!npm install -g localtunnel


changed 22 packages, and audited 23 packages in 1s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [13]:
%%writefile app.py

import streamlit as st
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import tokenizer_from_json
import json

# Load the model
@st.cache_resource
def load_model():
    return tf.keras.models.load_model('/content/drive/MyDrive/amazon reviews/review_amazon_sentiment5.h5')

model = load_model()

# Load the tokenizer
with open('/content/drive/MyDrive/amazon reviews/tokenizer.json', 'r') as f:
    data = json.load(f)

data_str = json.dumps(data)
tokenizer = tokenizer_from_json(data_str)

# Parameters
num_words = 15000
max_tokens = 166

# Streamlit app
st.title("Sentiment Analysis with LSTM")

# Section 1: Single Text Sentiment Analysis
st.header("Single Text Sentiment Classification")
input_text = st.text_area("Enter the text to classify", "")

if st.button("Classify Sentiment"):
    if input_text:
        # Tokenize and pad the input text
        input_sequence = tokenizer.texts_to_sequences([input_text])
        input_padded = pad_sequences(input_sequence, maxlen=max_tokens)

        # Predict sentiment
        prediction = model.predict(input_padded)[0][0]
        sentiment = "Negative" if prediction > 0.5 else "Positive"

        st.write(f"Predicted Sentiment: **{sentiment}**")
        st.write(f"Probability of being negative: {prediction:.2f}")

# Section 2: Batch Sentiment Analysis
st.header("Batch Sentiment Classification")
uploaded_file = st.file_uploader("Upload a CSV file", type=["csv"])

if uploaded_file is not None:
    # Load the CSV
    df = pd.read_csv(uploaded_file)

    # Ensure the dataframe has the correct columns
    if 'text' in df.columns:
        st.write("Analyzing data...")

        # Tokenize and pad the text data (No need to fit the tokenizer again)
        sequences = tokenizer.texts_to_sequences(df['text'].values)
        padded_sequences = pad_sequences(sequences, maxlen=max_tokens)

        # Predict sentiment
        predictions = model.predict(padded_sequences)
        df['sentiment_score'] = predictions
        df['sentiment'] = df['sentiment_score'].apply(lambda x: "Negative" if x > 0.5 else "Positive")

        # Display the dataframe
        st.write(df.head())

        # Download the resulting dataframe
        csv = df.to_csv(index=False).encode('utf-8')
        st.download_button(
            label="Download sentiment analysis as CSV",
            data=csv,
            file_name='sentiment.csv',
            mime='text/csv',
        )
    else:
        st.error("CSV must have a 'text' column.")



Overwriting app.py


In [14]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com  # ip is your password

34.125.133.211
your url is: https://giant-pumas-relate.loca.lt


# Trying out model in notebook before streamlit

In [ ]:
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import json

# Load the tokenizer JSON content as a string
with open('/content/drive/MyDrive/amazon reviews/tokenizer.json', 'r') as f:
    data = json.load(f)  # This gives you a dictionary

# Convert the dictionary back to a JSON string
data_str = json.dumps(data)

# Use the string to load the tokenizer
tokenizer = tokenizer_from_json(data_str)


# load model
model = tf.keras.models.load_model('/content/drive/MyDrive/amazon reviews/review_amazon_sentiment.h5')


# new data
review1 = "I hated this product, never buying it again!"
review2 = "beautiful! fast shipping and a responsive seller"
review3 = "garbage product, no one should sell such thing"
review4 = "great price for a product like this, definitely buying it again"

reviews = [review1, review2, review3, review4]


# apply tokenization
num_words = 15000
max_tokens = 162

tokens = tokenizer.texts_to_sequences(reviews)
tokens_pad = pad_sequences(tokens, maxlen=max_tokens)
tokens_pad.shape

In [ ]:
# prediction

 (model.predict(tokens_pad) >0.5).astype("int")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 983ms/step


array([[1],
       [0],
       [1],
       [0]])